In [1]:
import pandas as pd
import nltk
import spacy
import gensim
import seaborn as sb
from gensim import corpora, models, similarities
from spacy.lang.en import English
from nltk.corpus import wordnet as wn

In [2]:
data=pd.read_csv("RaleighCity_original_data.csv",)

In [3]:
data.head()

,Unnamed: 0,page_number,word
0,1,1,adopted
1,2,1,budget
2,3,1,fy
3,4,1,strengthening
4,5,1,our


In [4]:
data.drop(data.columns[0], axis=1)

,page_number,word
0,1,adopted
1,1,budget
2,1,fy
3,1,strengthening
4,1,our
5,1,foundation
6,1,in
7,1,workforce
8,1,operations
9,1,and


In [5]:
data.describe()

,Unnamed: 0,page_number
count,72573.00000,72573.000000
mean,36287.00000,162.375815
std,20950.16488,90.118814
min,1.00000,1.000000
25%,18144.00000,85.000000
50%,36287.00000,171.000000
75%,54430.00000,231.000000
max,72573.00000,317.000000


In [6]:
#cleaning 
spacy.load('en')
parser=English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.orth_.startswith('http'):
            lda_tokens.append('com')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [7]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\messi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
def get_lemma(word):
    lemma=wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

In [9]:
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [10]:
#filtering stopwords
nltk.download('stopwords')
stop=set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\messi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
#preparing for lda
def prepare_lda(text):
    tokens=tokenize(text)
    tokens=[token for token in tokens if len(token)>4]
    tokens=[token for token in tokens if token not in stop]
    tokens=[get_lemma(token) for token in tokens]
    return tokens

In [12]:
#creating tokens
import random
text_data=[]
with open('GuilfordCounty_original_data.csv') as f:
    for line in f:
        tokens=prepare_lda(line)
        if random.random()>.99:
            print(tokens)
            text_data.append(tokens)

['page_number","word']
['292",3,"theme']
['360",3,"beth']
['532",7,"development']
['597",7,"support']
['703",8,"extension']
['907",11,"honorable']
['949",11,"statutes']
['1101",11,"february']
['1193",11,"approximately']
['1244",12,"household']
['1246",12,"indicative']
['1334",12,"section']
['1373",12,"plan']
['1387",12,"to']
['1423",12,"in']
['1451",12,"county']
['1480",12,"state']
['1632",13,"to']
['1943",14,"tax']
['2003",14,"the']
['2008",14,"due']
['2176",15,"change']
['2281",15,"through']
['2398",15,"the']
['2404",15,"to']
['2440",15,"the']
['2504",15,"impact']
['2541",16,"charges']
['2586",16,"state']
['2650",16,"countys']
['2725",16,"and']
['2813",16,"to']
['2834",16,"fees']
['2862",16,"counties']
['3074",17,"chg']
['3156",17,"s']
['3178",17,"ns']
['3340",18,"rate']
['3409",18,"funding']
['3464",18,"board']
['3485",18,"rankings']
['3575",18,"allocations']
['3595",18,"retain']
['3763",19,"the']
['3839",19,"additional']
['3934",19,"for']
['3971",19,"and']
['3976",19,"coordinated']

['41311",188,"for']
['41377",188,"federal']
['41435",188,"county']
['41550",188,"professional']
['41649",189,"inducting']
['41719",189,"rank']
['41760",189,"project']
['42004",190,"population']
['42138",191,"restoring']
['42292",191,"budget']
['42333",191,"mileage']
['42535",193,"satisfied']
['42631",193,"fy']
['42792",195,"security']
['42855",195,"departmental']
['43084",196,"police']
['43125",196,"services']
['43128",196,"the']
['43160",196,"panic']
['43220",196,"life']
['43243",197,"response']
['43252",197,"including']
['43378",199,"waste']
['43525",199,"the']
['43619",199,"budget']
['43732",200,"funds']
['43757",200,"amended']
['43802",201,"interim']
['43858",201,"supplies']
['43922",201,"opportunities']
['43990",202,"environment']
['44057",202,"program']
['44327",203,"rebuild']
['44399",203,"engage']
['44684",204,"to']
['44950",204,"life']
['45002",204,"well']
['45110",205,"the']
['45130",205,"goals']
['45155",205,"our']
['45246",206,"total']
['45377",207,"to']
['45426",207,"a']
[

['78544",370,"facilities']
['78744",371,"complete']
['78815",371,"total']
['78844",372,"start']
['78978",372,"courts']
['79117",373,"the']
['79174",374,"code']
['79177",374,"high']
['79212",374,"replacement']
['79293",374,"fy']
['79325",374,"a']
['79384",375,"of']
['79523",376,"replacement']
['79854",378,"extend']
['79888",378,"constructionrepair']
['80106",379,"february']
['80117",379,"ballot']
['80401",381,"bonds']
['80618",382,"dudley']
['80630",383,"budget']
['80669",383,"expense']
['80716",383,"elementry']
['80811",384,"sch']
['80849",384,"expense']
['80902",384,"high']
['81026",385,"western']
['81087",385,"revenue']
['81120",385,"parks']
['81212",386,"park']
['81219",386,"open']
['81353",389,"law']
['81507",389,"public']
['81652",389,"to']
['81692",389,"county']
['81705",389,"an']
['81752",389,"at']
['81800",390,"the']
['81809",390,"in']
['81888",390,"can']
['81942",390,"functions']
['82103",390,"of']
['82236",390,"capital']
['82539",391,"countywide']
['82551",391,"to']
['82579",

In [13]:
#creating dictionary
from gensim import corpora
dictionary=corpora.Dictionary(text_data)
corpus= [dictionary.doc2bow(text) for text in text_data]
import pickle 
pickle.dump(corpus, open('corpus.pkl', 'wb'))

In [14]:
#example
ldamodel=gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=6)
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} word: {}'.format(idx, topic))

Topic: 0 word: 0.006*"56015",251,"backtoschool" + 0.006*"43990",202,"environment" + 0.006*"71670",328,"major" + 0.006*"79293",374,"fy" + 0.006*"2003",14,"the" + 0.006*"45110",205,"the" + 0.006*"58641",261,"explore" + 0.006*"53405",241,"administration" + 0.006*"83710",394,"specified" + 0.006*"69443",315,"management"
Topic: 1 word: 0.006*"8914",36,"excess" + 0.006*"20176",95,"service" + 0.006*"23641",110,"method" + 0.006*"3340",18,"rate" + 0.006*"36415",169,"continues" + 0.006*"34331",159,"december" + 0.006*"86538",430,"from" + 0.006*"15788",72,"countys" + 0.006*"79523",376,"replacement" + 0.006*"4656",21,"increase"
Topic: 2 word: 0.006*"17593",81,"funds" + 0.006*"32625",152,"adjustments" + 0.006*"89420",459,"following" + 0.006*"49817",223,"each" + 0.006*"53037",239,"distinguished" + 0.006*"35857",164,"trending" + 0.006*"54182",244,"services" + 0.006*"61550",271,"budget" + 0.006*"4012",19,"include" + 0.006*"45998",209,"decline"
Topic: 3 word: 0.006*"4105",20,"the" + 0.006*"23984",113,"re

In [15]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\Users\messi\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
